In [9]:
import glob, codecs, StringIO
import base64
from ToolsMoodleXML import *
import ToolsMoodleXML
from PIL import Image

In [10]:
repertoire="/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K1/"
nomImages=glob.glob(repertoire+"Images/16-K1-PREVIEW*.png")
nomImages=sorted(nomImages,key=lambda x: len(x))
nomTraductions="Images.txt"

In [11]:
categorie=u"Kalaba16/K1/Écriture/DDT/"

- Choisir un thème entre *Phonétique* et *Traductions*

In [12]:
theme=u"Phonétique"
theme=u"Traductions"
questions=XMLSeries(categorie+theme)

In [13]:
traductionsLignes=codecs.open(repertoire+nomTraductions,"r",encoding="utf8").readlines()
prononciations=[]
traductions=[]
for t in traductionsLignes:
    (prononciation,traduction)=t.strip().split(";")
    prononciations.append(prononciation)
    traductions.append(traduction)

In [14]:
def encodeImage(image):
    ioImage=StringIO.StringIO()
    resizedImage=Image.new(mode='RGBA', size=(600, image.size[1]), color=(0,0,0,0))
    resizedImage.paste(image, (0, 0))
    resizedImage.save(ioImage,"PNG")
    result=base64.b64encode(ioImage.getvalue())
    return result

In [15]:
groupeNum=5
phraseNum=0

for num,filenames in enumerate(zip(*(iter(nomImages),) * groupeNum)):
    consigne=""
    images = [Image.open(filename) for filename in filenames]
    for numImage,image in enumerate([encodeImage(f) for f in images]):
        ligneImage=u'<p>Écriture : <img alt="Embedded Image" src="data:image/png;base64,%s"/></p>'%image
        ligneReponse=u'<p>Prononciation : [[%d]]'%(numImage+1)        
        consigne+=ligneImage+ligneReponse
    pieces=[]
    for i in range(groupeNum):
        if theme=="Traductions":
            pieces.append(traductions[phraseNum])
        else:
            pieces.append(prononciations[phraseNum])
        phraseNum+=1
    ddt=DragDropText(u"Écriture %02d"%(num+1),
                      consigne,
                      pieces
                     )
    questions.addExercice(ddt)

In [16]:
xmlOutput=MoodleXML()
xmlOutput.addQuiz(questions.getSeries())

In [17]:
xmlFichier=repertoire+nomTraductions.replace(".txt","-%s.xml"%theme)
with codecs.open(xmlFichier, 'w',encoding="utf8") as output:
    output.write(xmlOutput.getXML())